In [56]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

In [57]:
warnings.filterwarnings('ignore')

In [45]:
sns.set_theme(style='whitegrid')

#### Define global variables

In [50]:
NAMES = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Shirt']

## Load the data

In [46]:
train = np.load('fashion_train.npy')
test = np.load('fashion_test.npy')

X_train, y_train = train[:, :784], train[:, 784]
X_test, y_test = test[:, :784], test[:, 784]

# Exploratory Data Analysis

#### Numerical variables

In [83]:
#shape of the data
print('Shape of the training data: ', X_train.shape)
print('Shape of the testing data: ', X_test.shape)

#mean, std, min, max
print('Mean of the training data: ', X_train.mean())
print('Std of the training data: ', X_train.std())
print('Min of the training data: ', X_train.min())
print('Max of the training data: ', X_train.max())


Shape of the training data:  (10000, 784)
Shape of the testing data:  (5000, 784)
Mean of the training data:  77.02673711734694
Std of the training data:  89.9969501937854
Min of the training data:  0
Max of the training data:  255


#### What is the distribution of the target variable?

In [71]:
def plot_y_distribution(y):
    classes, classes_counts = np.unique(y, return_counts=True)

    fig, ax = plt.subplots(figsize=(10, 6))
    sns.barplot(x=classes, y=classes_counts, ax=ax);
    ax.set_title('Class distribution', fontsize=16, fontweight='bold');
    ax.set_ylabel('Count');
    ax.set_xticklabels(NAMES);


#plot_y_distribution(y_train)

#### What are the properties of images in the dataset?
We investigate mean, median, and stdev of the pixel values in the images.

In [81]:
def plot_images(X, measure='mean'):
    classes = np.unique(y_train)
    fig, ax = plt.subplots(1, len(classes), figsize=(20, 8))
    for c in classes:
        imgs = X[y_train == c]

        if measure == 'mean': av_img = np.mean(imgs, axis=0).reshape(28, 28)
        elif measure == 'median': av_img = np.median(imgs, axis=0).reshape(28, 28)
        elif measure == 'std': av_img = np.std(imgs, axis=0).reshape(28, 28)

        ax[c].imshow(av_img, cmap='gray')
        ax[c].set_title(NAMES[c], fontsize=16, fontweight='bold')


#plot_images(X_train)
#plot_images(X_train, measure='std')
#plot_images(X_train, measure='median')